In [1]:
import pandas as pd
import numpy as np
import json
import ast

In [2]:
Films = pd.DataFrame
Films = pd.read_csv("C:/Users/Fede/Documents/Henry/Labs/Fede/movies_dataset (1).csv")


C:\Users\Fede\AppData\Local\Temp\ipykernel_14136\2699400193.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  Films = pd.read_csv("C:/Users/Fede/Documents/Henry/Labs/Fede/movies_dataset (1).csv")


In [3]:
# Leer solo las columnas 'cast' y 'crew' del archivo credits.csv
df2 = pd.read_csv("C:/Users/Fede/Documents/Henry/Labs/Fede/credits (1).csv", usecols=['cast', 'crew'])

# Agregar las columnas 'cast' y 'crew' del DataFrame df2 al DataFrame df1
Films['cast'] = df2['cast']
Films['crew'] = df2['crew']


In [4]:

def extraer_nombre_actor(cast):
    if pd.isnull(cast):
        return np.nan
    else:
        try:
            lista_de_actores = ast.literal_eval(cast)
            nombre_de_actores = [actor['name'] for actor in lista_de_actores]
            return ', '.join(nombre_de_actores)  # Devuelve los nombres y apellidos de los actores separados por coma
        except (ValueError, TypeError):
            return np.nan



In [5]:
# Aplicar la función extraer_nombre_actor a la columna 'cast'
Films['actores'] = Films['cast'].apply(extraer_nombre_actor)

In [6]:
Films.drop(['cast'], axis=1, inplace=True)

In [7]:
def extraer_director(director):
    if pd.isnull(director):
        return np.nan
    else:
        try:
            lista_de_directores = ast.literal_eval(director)
            for member in lista_de_directores:
                if member['job'] == 'Director':
                    return member['name']
            return np.nan
        except (ValueError, TypeError):
            return np.nan


In [8]:
# Aplicar la función extraer_director a la columna 'crew' y crear la columna 'director'
Films['director'] = Films['crew'].apply(extraer_director)

In [9]:
Films.drop('crew', axis=1, inplace=True)

In [10]:
# Función personalizada para desanidar la columna 'belongs_to_collection'
def extract_collection_name(collection):
    try:
        return ast.literal_eval(collection)['name']
    except (ValueError, TypeError):
        return None


In [11]:
# Aplicar la función personalizada a la columna 'belongs_to_collection'
Films['belongs_to_collection'] = Films['belongs_to_collection'].apply(extract_collection_name)


In [12]:
# Función para desanidar la columna 'spoken_languages' y extraer el nombre del idioma
def extract_language_name(languages):
    if pd.isnull(languages):
        return pd.NA
    try:
        languages_list = eval(languages)
        language_names = [lang['name'] for lang in languages_list]
        return ", ".join(language_names)
    except (ValueError, TypeError):
        return pd.NA

In [13]:
# Aplicar la función para desanidar 'spoken_languages' y obtener el nombre del idioma
Films['spoken_languages'] = Films['spoken_languages'].apply(extract_language_name)


In [14]:
# Función para desanidar la columna 'genres' y extraer los nombres de los géneros
def extract_genre_names(genres):
    if pd.isnull(genres):
        return pd.NA
    try:
        genres_list = eval(genres)
        genre_names = [genre['name'] for genre in genres_list]
        return ", ".join(genre_names)
    except (ValueError, TypeError):
        return pd.NA


In [15]:
# Aplicar la función para desanidar 'genres' y extraer los nombres de los géneros
Films['genres'] = Films['genres'].apply(extract_genre_names)

In [16]:
# Función para desanidar la columna 'production_countries' y extraer los nombres de los países de producción
def extract_country_names(countries):
    if pd.isnull(countries):
        return pd.NA
    try:
        countries_list = eval(countries)
        country_names = [country['name'] for country in countries_list]
        return ", ".join(country_names)
    except (ValueError, TypeError):
        return pd.NA


In [17]:
# Aplicar la función para desanidar 'production_countries' y obtener los nombres de los países de producción
Films['production_countries'] = Films['production_countries'].apply(extract_country_names)

In [18]:
# Función para desanidar la columna 'production_companies' y extraer los nombres de las compañías de producción
def extract_production_company_names(companies):
    if pd.isnull(companies):
        return pd.NA
    try:
        companies_list = eval(companies)
        company_names = [company['name'] for company in companies_list]
        return ", ".join(company_names)
    except (ValueError, TypeError):
        return pd.NA


In [19]:
# Aplicar la función para desanidar 'production_companies' y obtener los nombres de las compañías de producción
Films['production_companies'] = Films['production_companies'].apply(extract_production_company_names)

In [20]:
# Rellenar los valores nulos en la columna "budget" con 0
Films['budget'].fillna(0, inplace=True)

In [21]:
# Rellenar los valores nulos en la columna "revenue" con 0
Films['revenue'].fillna(0, inplace=True)

In [22]:
# Eliminar los valores nulos del campo 'release_date'
Films.dropna(subset=['release_date'], inplace=True)

In [23]:
# Convertir la columna 'release_date' a objetos de fecha y manejar los valores no válidos
Films['release_date'] = pd.to_datetime(Films['release_date'], errors='coerce')

# Eliminar las filas que contienen valores NaT (no válidos)
Films.dropna(subset=['release_date'], inplace=True)

# Formatear las fechas en el formato 'AAAA-mm-dd'
Films['release_date'] = Films['release_date'].dt.strftime('%Y-%m-%d')


In [24]:
# Función para calcular el retorno de inversión
def calculate_return(row):
    revenue = pd.to_numeric(row['revenue'], errors='coerce')
    budget = pd.to_numeric(row['budget'], errors='coerce')
    if pd.notna(revenue) and pd.notna(budget) and budget != 0:
        return revenue / budget
    return 0

# Crear la nueva columna 'return' aplicando la función de cálculo
Films['return'] = Films.apply(calculate_return, axis=1)

In [25]:
# Lista de columnas a eliminar
columnas_sin_utilizar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'id', 'status', 'tagline']

# Eliminar las columnas que no serán utilizadas
Films.drop(columns=columnas_sin_utilizar, inplace=True)

In [26]:
# Convertir la columna release_date a tipo datetime si no lo está ya
Films['release_date'] = pd.to_datetime(Films['release_date'])

# Definir el rango de fechas para filtrar
fecha_inicio = pd.to_datetime('1979-12-01')
fecha_fin = pd.to_datetime('2022-12-01')

# Filtrar las filas basadas en el rango de fechas
Films = Films[(Films['release_date'] >= fecha_inicio) & (Films['release_date'] <= fecha_fin)]


In [27]:
# Lista de columnas a eliminar
columnas_sin_utilizar = ['vote_average', 'vote_count']

# Eliminar las columnas que no serán utilizadas
Films.drop(columns=columnas_sin_utilizar, inplace=True)

In [28]:
Films.to_csv('Films.csv', index=False)